13) MOCK DATASETS (don't use the AM207 ones):

    a) Use some ISM parameters that represent different components of the ISM e.g. leave ISM parameters at their prior values and each time vary one by 2 sigma. one bursty SFR environment (SFR - 2sigma), one leaky (x_out + 2), one low SFE environment. And then see how well you can retrieve those. You will also see, that the ISM parameters are correlated (the signal will be redistributed into ather ISM parameters). I suggest it this way because in a paper you can plot the different tracks, name them (bursty, leaky etc) and plot the mock star abundances at specific times. And then you can show how well you can retrieve them. Of course you will have to add observational uncertainties to your predicted abundances. You can test with very low observational uncertainties, but a fair test would be to use typical observational uncertainties. This doesn't need any error parameters per element.
    
    b) In a last step you will need to do the inference with a wrong yield set as well (only then the error parameters per element will also be needed).



In [2]:
%pylab inline
import os
os.chdir('/home/oliver/ChempyMulti/')
from Chempy.parameter import ModelParameters
from Chempy.cem_function import single_timestep_chempy
a=ModelParameters()

Populating the interactive namespace from numpy and matplotlib


First we choose some SSP parameters:

In [69]:
from scipy.stats import norm
global_params = norm.rvs(loc=a.p0[:2],scale=[0.3,0.3]).round(2)

In [71]:
global_params

array([-2.45, -2.86])

In [72]:
# Use 5 time datapoints
times = np.linspace(2,12,5)

ism_priors = a.p0[2:]
ism_prior_widths = [0.3,0.1,0.1]

# Create 4 sets of ISM parameters
ism_params = np.ones((4,1))*ism_priors
ism_params[1,0]-=2*ism_prior_widths[0] # low SFE
ism_params[2,1]-=2*ism_prior_widths[1] # bursty SFR
ism_params[3,2]+=2*ism_prior_widths[2] # leaky ISM

In [73]:
full_params = np.zeros([len(ism_params)*len(times),6])
for i in range(len(full_params)):
    full_params[i]=np.asarray(list(global_params)+list(ism_params[i//len(times)])+[times[i%len(times)]])

In [74]:

def runner(par):
    """Function to compute the Chempy predictions for each parameter set"""
    b=ModelParameters()
    output=single_timestep_chempy((par,b))
    if type(output)==float:
        if output==np.inf:
            del b
        outs=np.zeros(len(els))
    else: 
        abun=output[0]
        del b;
        outs=abun
    return outs


In [75]:
mock_abundances=np.asarray([runner(par) for par in full_params])

In [76]:
_,chempy_els=single_timestep_chempy((full_params[0],a))

Use observational errors as solar errors here:

In [77]:
sol_dat = np.load('Chempy/input/stars/Proto-sun_all.npy')
sol_names = sol_dat.dtype.names
all_sol_errors = sol_dat[1]

In [78]:
obs_errors = np.zeros(len(a.initial_neural_names))
for i,el in enumerate(a.initial_neural_names):
    for j,el_sol in enumerate(sol_names):
        if str(el)==str(sol_names[j]):
            obs_errors[i]=all_sol_errors[j]
            break
        if j==len(sol_names)-1:
            print('Element %s not found'%el)
mock_ordered_abundances = np.zeros([len(mock_abundances),len(a.initial_neural_names)])        
for i,el in enumerate(a.initial_neural_names):
    for j,el_chempy in enumerate(chempy_els):
        if str(el)==str(chempy_els[j]):
            mock_ordered_abundances[:,i]=mock_abundances[:,j]
            break
        if j==len(chempy_els)-1:
            print('Element %s not found'%el)
        

In [79]:
mock_errors = np.vstack([obs_errors for _ in range(len(mock_abundances))])

In [82]:
from scipy.stats import norm
time_mean = norm.rvs(loc=times,scale=0.5).round(2)
time_error = 0.5*np.ones_like(time_mean)
all_time_means = np.zeros(len(full_params))
all_time_stds = np.zeros(len(full_params))
for i in range(len(full_params)):
    all_time_means[i]=time_mean[i%len(times)]
    all_time_stds[i] = time_error[i%len(times)]

In [83]:
np.savez('ChempyMultiData/MockParams.npz',params=full_params,abundances=mock_ordered_abundances,
        time_mean = all_time_means, time_error = all_time_stds, abundance_error = mock_errors,elements=a.initial_neural_names)